In [7]:
# Re-run generator with bug fix: avoid random.randint with invalid ranges when remaining time < 5s.
import os
import json
import random
import uuid
from datetime import datetime, timedelta
import pandas as pd
from collections import defaultdict

random.seed(42)

OUT_DIR = r"C:\Users\12vat\engagement drop\datas"
os.makedirs(OUT_DIR, exist_ok=True)

NUM_USERS = 3000
NUM_CONTENT = 100
DAYS = 14
START_DATE = datetime(2025, 11, 10)
CONTENT_IDS = [f"MOV{100 + i}" for i in range(NUM_CONTENT)]
REGIONS = ["IN-North", "IN-South", "IN-East", "IN-West", "EU-West", "US-East", "APAC"]

users = []
for uid in range(1, NUM_USERS + 1):
    users.append({
        "user_id": uid,
        "device": random.choices(["android", "ios", "smart_tv", "laptop"], weights=[0.45,0.30,0.15,0.10])[0],
        "region": random.choice(REGIONS),
        "app_version": random.choice(["1.0","1.1","1.2","1.3"])
    })

region_multipliers = {}
for r in REGIONS:
    region_multipliers[r] = random.uniform(0.82, 1.0)
region_multipliers_df = pd.DataFrame(list(region_multipliers.items()), columns=["region","post_change_multiplier"])

def iso(ts):
    return ts.strftime("%Y-%m-%dT%H:%M:%SZ")

def generate_session_events(user, session_start_ts, avg_session_minutes, content_pool, bounce_prob):
    events = []
    events.append({
        "event_id": str(uuid.uuid4()),
        "user_id": user["user_id"],
        "event_type": "app_open",
        "content_id": None,
        "device": user["device"],
        "app_version": user["app_version"],
        "region": user["region"],
        "timestamp": iso(session_start_ts)
    })
    events.append({
        "event_id": str(uuid.uuid4()),
        "user_id": user["user_id"],
        "event_type": "home_impression",
        "content_id": None,
        "device": user["device"],
        "app_version": user["app_version"],
        "region": user["region"],
        "timestamp": iso(session_start_ts + timedelta(seconds=1))
    })
    if random.random() < bounce_prob:
        events.append({
            "event_id": str(uuid.uuid4()),
            "user_id": user["user_id"],
            "event_type": "app_close",
            "content_id": None,
            "device": user["device"],
            "app_version": user["app_version"],
            "region": user["region"],
            "timestamp": iso(session_start_ts + timedelta(seconds=random.randint(2,25)))
        })
        return events
    content = random.choice(content_pool)
    play_ts = session_start_ts + timedelta(seconds=random.randint(2,10))
    events.append({
        "event_id": str(uuid.uuid4()),
        "user_id": user["user_id"],
        "event_type": "video_play",
        "content_id": content,
        "device": user["device"],
        "app_version": user["app_version"],
        "region": user["region"],
        "timestamp": iso(play_ts)
    })
    session_seconds = max(30, int(random.gauss(avg_session_minutes*60, avg_session_minutes*30)))
    elapsed = 0
    last_ts = play_ts
    pauses = 0
    while elapsed < session_seconds:
        rem = session_seconds - elapsed
        # if remaining less than 5s, just advance to end
        if rem <= 5:
            delta = rem
        else:
            delta = random.randint(5, min(30, rem))
        if delta <= 0:
            break
        elapsed += delta
        last_ts = last_ts + timedelta(seconds=delta)
        p = random.random()
        if p < 0.05:
            events.append({
                "event_id": str(uuid.uuid4()),
                "user_id": user["user_id"],
                "event_type": "buffer_start",
                "content_id": content,
                "device": user["device"],
                "app_version": user["app_version"],
                "region": user["region"],
                "timestamp": iso(last_ts)
            })
            last_ts = last_ts + timedelta(seconds=random.randint(1,8))
            events.append({
                "event_id": str(uuid.uuid4()),
                "user_id": user["user_id"],
                "event_type": "buffer_end",
                "content_id": content,
                "device": user["device"],
                "app_version": user["app_version"],
                "region": user["region"],
                "timestamp": iso(last_ts)
            })
        elif p < 0.12:
            pauses += 1
            events.append({
                "event_id": str(uuid.uuid4()),
                "user_id": user["user_id"],
                "event_type": "video_pause",
                "content_id": content,
                "device": user["device"],
                "app_version": user["app_version"],
                "region": user["region"],
                "timestamp": iso(last_ts)
            })
            last_ts = last_ts + timedelta(seconds=random.randint(2,30))
            events.append({
                "event_id": str(uuid.uuid4()),
                "user_id": user["user_id"],
                "event_type": "video_resume",
                "content_id": content,
                "device": user["device"],
                "app_version": user["app_version"],
                "region": user["region"],
                "timestamp": iso(last_ts)
            })
        elif p < 0.18:
            events.append({
                "event_id": str(uuid.uuid4()),
                "user_id": user["user_id"],
                "event_type": "video_seek",
                "content_id": content,
                "device": user["device"],
                "app_version": user["app_version"],
                "region": user["region"],
                "timestamp": iso(last_ts),
                "seek_seconds": random.randint(-30,30)
            })
    if random.random() < 0.6:
        events.append({
            "event_id": str(uuid.uuid4()),
            "user_id": user["user_id"],
            "event_type": "video_complete",
            "content_id": content,
            "device": user["device"],
            "app_version": user["app_version"],
            "region": user["region"],
            "timestamp": iso(last_ts + timedelta(seconds=1))
        })
    else:
        events.append({
            "event_id": str(uuid.uuid4()),
            "user_id": user["user_id"],
            "event_type": "video_stop",
            "content_id": content,
            "device": user["device"],
            "app_version": user["app_version"],
            "region": user["region"],
            "timestamp": iso(last_ts + timedelta(seconds=random.randint(1,10)))
        })
    events.append({
        "event_id": str(uuid.uuid4()),
        "user_id": user["user_id"],
        "event_type": "app_close",
        "content_id": None,
        "device": user["device"],
        "app_version": user["app_version"],
        "region": user["region"],
        "timestamp": iso(last_ts + timedelta(seconds=random.randint(5,40)))
    })
    return events

summary = defaultdict(int)
all_preview = []
for day_idx in range(DAYS):
    day_date = START_DATE + timedelta(days=day_idx)
    filename = os.path.join(OUT_DIR, f"day_{day_idx+1:02d}.json")
    with open(filename, "w") as fh:
        day_events = 0
        for user in users:
            baseline_sessions = random.choice([3,4,4,5])
            if day_idx < 7:
                sessions = baseline_sessions
                avg_session_minutes = random.uniform(12,25)
                content_pool = CONTENT_IDS
                bounce_prob = 0.08
            else:
                mult = region_multipliers[user["region"]]
                sessions = max(1, int(round(baseline_sessions * mult)))
                if random.random() < 0.7:
                    niche_pool = random.sample(CONTENT_IDS, k=random.randint(10,15))
                    content_pool = niche_pool
                else:
                    content_pool = CONTENT_IDS
                avg_session_minutes = random.uniform(6,14)
                bounce_prob = min(0.6, 0.12 + (1.0 - mult) * 1.0)
            for s in range(sessions):
                slot = random.choices(["morning","afternoon","evening","night"], weights=[0.2,0.2,0.45,0.15])[0]
                if slot == "morning":
                    start_hour = random.randint(7,10)
                elif slot == "afternoon":
                    start_hour = random.randint(12,15)
                elif slot == "evening":
                    start_hour = random.randint(18,22)
                else:
                    start_hour = random.randint(23,23) if random.random() < 0.6 else random.randint(0,3)
                start_min = random.randint(0,59)
                start_sec = random.randint(0,59)
                session_start_ts = datetime(day_date.year, day_date.month, day_date.day, start_hour, start_min, start_sec)
                events = generate_session_events(user, session_start_ts, avg_session_minutes, content_pool, bounce_prob)
                for ev in events:
                    fh.write(json.dumps(ev) + "\n")
                    day_events += 1
                    if len(all_preview) < 50:
                        all_preview.append(ev)
        summary[f"day_{day_idx+1:02d}"] = day_events

summary_df = pd.DataFrame(list(summary.items()), columns=["day","events"])
summary_df.to_csv("/mnt/data/engagement_logs/summary_by_day.csv", index=False)
preview_df = pd.DataFrame(all_preview)
preview_df.to_csv("/mnt/data/engagement_logs/preview_events.csv", index=False)
region_multipliers_df.to_csv("/mnt/data/engagement_logs/region_multipliers.csv", index=False)


print("Files generated under /mnt/data/engagement_logs/")
print("Sample summary (first 7 rows):")
print(summary_df.head().to_string(index=False))
print("\nRegion multipliers (impact per region):")
print(region_multipliers_df.to_string(index=False))


Files generated under /mnt/data/engagement_logs/
Sample summary (first 7 rows):
   day  events
day_01  271634
day_02  276761
day_03  272792
day_04  271754
day_05  273318

Region multipliers (impact per region):
  region  post_change_multiplier
IN-North                0.888608
IN-South                0.954272
 IN-East                0.974039
 IN-West                0.891401
 EU-West                0.821933
 US-East                0.840643
    APAC                0.985085


In [13]:
pip install psycopg2-binary

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.7 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.7 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.7 MB 882.6 kB/s eta 0:00:03
   ------- -------------------------------- 0.5/2.7 MB 882.6 kB/s eta 0:00:03
   ----------- ---------------------------- 0.8/2.7 MB 670.4 kB/s eta 0:00:03
   --------------- ------------------------ 1.0/2.7 MB 729.5 kB/s eta 0:00:03
   ------------------- -------------------- 1.3/2.7 MB 871.6 kB/s eta 0:00:02
   ----------------------- ---------------- 1.6/2.7 MB 942.3 kB/s eta 0:00:02
   ----------------------- ---------------- 1.6/2.7 MB 942.3 kB/s eta 0:00:02
   ----------------------- ---------------- 1.6/2.7 MB 942.3 kB/s eta 0:00:02
   --------------------------- ------------ 1.8/2.7 MB 786.4 kB/s eta 0:00:02
   -------------------

In [15]:
import psycopg2
import psycopg2.extras 
import json
import glob
import os

# --- 🎯 CONFIGURATION: UPDATE THESE ---
DB_NAME = "engagement drop"
DB_USER = "postgres"
DB_PASSWORD = "7860"
DB_HOST = "localhost"
LOG_DIR = r"C:\Users\12vat\engagement drop\datas" # Use your local path where day_*.json files are saved
# -----------------------------------

def import_json_logs():
    conn = None
    try:
        # 1. Connect to the PostgreSQL database
        conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST)
        cur = conn.cursor()
        print("Connected to PostgreSQL successfully.")

        # 2. SQL INSERT statement (maps Python fields to SQL columns)
        insert_sql = """
        INSERT INTO engagement_events (event_id, user_id, event_type, timestamp, content_id, device, app_version, region, seek_seconds)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (event_id) DO NOTHING;
        """
        
        total_events_inserted = 0
        
        # 3. Process each JSON Lines file
        for filename in glob.glob(os.path.join(LOG_DIR, "day_*.json")):
            events_to_insert = []
            with open(filename, 'r') as f:
                for line in f:
                    try:
                        record = json.loads(line)
                        # Extract and structure data for the INSERT command
                        event_data = (
                            record.get("event_id"),
                            record.get("user_id"),
                            record.get("event_type"), 
                            record.get("timestamp"),
                            record.get("content_id"),
                            record.get("device"),
                            record.get("app_version"),
                            record.get("region"),
                            record.get("seek_seconds") # seek_seconds is optional (None if not present)
                        )
                        events_to_insert.append(event_data)
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON in {filename}: {e}")
                
                # Execute batch insert for speed
                psycopg2.extras.execute_batch(cur, insert_sql, events_to_insert, page_size=5000)
                total_events_inserted += len(events_to_insert)
                print(f"Imported {len(events_to_insert)} events from {os.path.basename(filename)}")

        conn.commit()
        print(f"\n--- Import Complete. Total events inserted: {total_events_inserted} ---")

    except Exception as error:
        print(f"Error during database import: {error}")
        if conn:
            conn.rollback()
    finally:
        if conn:
            cur.close()
            conn.close()

if __name__ == "__main__":
    import_json_logs()

Connected to PostgreSQL successfully.
Imported 271634 events from day_01.json
Imported 276761 events from day_02.json
Imported 272792 events from day_03.json
Imported 271754 events from day_04.json
Imported 273318 events from day_05.json
Imported 273099 events from day_06.json
Imported 274287 events from day_07.json
Imported 143552 events from day_08.json
Imported 144097 events from day_09.json
Imported 143600 events from day_10.json
Imported 143314 events from day_11.json
Imported 141049 events from day_12.json
Imported 144417 events from day_13.json
Imported 144003 events from day_14.json

--- Import Complete. Total events inserted: 2917677 ---
